In [1]:
def predict_emotion_from_image(image, model, label_encoder, transform, device, enhance_quality=True):
    """
    Görüntüden duygu tahmini yapar (girdi hem numpy array hem de PIL.Image olabilir)

    Args:
        image: numpy array (BGR) veya PIL.Image
        model: Eğitilmiş model
        label_encoder: Label encoder
        transform: Görüntü dönüşümleri
        device: Torch device
        enhance_quality: Görüntü kalitesini artır (True/False)

    Returns:
        Dictionary containing prediction results
    """
    try:
        # Eğer numpy array (OpenCV) geldiyse → PIL'e çevir
        if isinstance(image, np.ndarray):
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image)

        # Görüntü kalitesini artır (isteğe bağlı)
        if enhance_quality:
            image = enhance_image_quality(image)

        # Model için transform uygula
        input_tensor = transform(image).unsqueeze(0).to(device)

        # Model prediction
        model.eval()
        with torch.no_grad():
            outputs = model(input_tensor)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            predicted_idx = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0, predicted_idx].item()

        predicted_emotion = label_encoder.inverse_transform([predicted_idx])[0]

        # Tüm olasılıkları sırala
        all_probs = probabilities[0].cpu().numpy()
        emotion_labels = label_encoder.classes_
        sorted_indices = np.argsort(all_probs)[::-1]
        sorted_probabilities = [(emotion_labels[i], all_probs[i]) for i in sorted_indices]

        result = {
            'predicted_emotion': predicted_emotion,
            'confidence': confidence,
            'sorted_probabilities': sorted_probabilities,
            'enhanced_image': image if enhance_quality else None
        }

        return result

    except Exception as e:
        print(f"Error in prediction: {str(e)}")
        return None


In [2]:
def apply_unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=1.5, threshold=0):
    """Gelişmiş unsharp masking"""
    if isinstance(image, Image.Image):
        img_array = np.array(image)
    else:
        img_array = image

    # Gaussian blur uygula
    blurred = cv2.GaussianBlur(img_array, kernel_size, sigma)

    # Unsharp mask oluştur
    unsharp_mask = cv2.addWeighted(img_array, 1.0 + amount, blurred, -amount, 0)

    # Threshold uygula (opsiyonel)
    if threshold > 0:
        low_contrast_mask = np.absolute(img_array - blurred) < threshold
        np.copyto(unsharp_mask, img_array, where=low_contrast_mask)

    return Image.fromarray(unsharp_mask)

In [3]:
from PIL import Image, ImageFile, ImageEnhance
def enhance_image_quality(image):
    """Comprehensive image enhancement"""
    # Sharpening uygula
    sharpened = apply_unsharp_mask(image, sigma=1.2, amount=1.3)

    # Contrast enhancement
    enhancer = ImageEnhance.Contrast(sharpened)
    contrast_enhanced = enhancer.enhance(1.1)

    # Sharpness enhancement (PIL built-in)
    enhancer = ImageEnhance.Sharpness(contrast_enhanced)
    final_enhanced = enhancer.enhance(1.2)

    return final_enhanced

In [4]:
from torchvision import transforms
transform_predict = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
import pickle

with open("label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

print("Sınıflar:", le.classes_)

Sınıflar: ['anger' 'happiness' 'neutrality' 'sadness' 'surprise']


In [6]:
with open("model_info_20250825_1742.pkl", "rb") as f:
    model_info = pickle.load(f)

print(model_info)

{'model_state_dict': OrderedDict([('features.0.0.weight', tensor([[[[ 9.3470e-01,  1.1869e+00,  9.8474e-01],
          [ 9.7756e-01,  1.2279e+00,  9.2944e-01],
          [ 1.2869e-01,  1.8335e-01,  2.3656e-02]],

         [[-7.1979e-01, -9.3504e-01, -8.7627e-01],
          [-7.7244e-01, -9.7241e-01, -8.1216e-01],
          [ 7.8490e-03, -9.0685e-02, -2.6237e-02]],

         [[ 3.3178e-03, -3.7360e-02, -9.2698e-02],
          [-3.5389e-02, -6.0272e-02, -9.8414e-02],
          [ 5.0979e-02,  1.9786e-02,  1.8379e-02]]],


        [[[-3.6820e-02, -2.3139e-02,  1.4284e-03],
          [ 3.4145e-01,  6.9242e-01,  4.1152e-01],
          [-3.3092e-01, -6.5212e-01, -3.5524e-01]],

         [[-1.1535e-03,  3.5697e-03,  7.1105e-03],
          [ 7.2458e-01,  1.5472e+00,  8.6126e-01],
          [-7.1552e-01, -1.5623e+00, -8.5459e-01]],

         [[-3.6598e-03,  1.5755e-02,  1.4657e-02],
          [ 1.3685e-01,  2.8022e-01,  1.6143e-01],
          [-1.2629e-01, -2.6898e-01, -1.5246e-01]]],


        

In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
import pickle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MobileNetV3-Small modeli oluştur
model = models.mobilenet_v3_small(pretrained=False)  # pretrained=False çünkü zaten eğitilmiş halini yükleyeceğiz
in_features = model.classifier[3].in_features        # genelde 1024 oluyor
model.classifier[3] = nn.Linear(in_features, len(le.classes_))

# Eğitilmiş ağırlıkları yükle
model.load_state_dict(torch.load("final_emotion_model_20250825_1742.pth", map_location=device))
model = model.to(device)
model.eval()

print("✅ Model yüklendi")
print("Sınıflar:", le.classes_)

✅ Model yüklendi
Sınıflar: ['anger' 'happiness' 'neutrality' 'sadness' 'surprise']


C:\Users\mibra\anaconda3\envs\fer-gpu\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\mibra\anaconda3\envs\fer-gpu\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\mibra\AppData\Local\Temp\ipykernel_18332\1367201119.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `w

In [8]:
import cv2

# Haarcascade XML dosyası (OpenCV’nin içinden gelir)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


In [11]:
import pyautogui
import time
import os
from datetime import datetime
import tkinter as tk
from tkinter import messagebox, simpledialog, ttk
import threading
import numpy as np

# Windows için ek kütüphaneler
try:
    import win32gui
    import win32ui
    import win32con
    import win32api
    from ctypes import windll
    from PIL import Image
    WINDOWS_AVAILABLE = True
except ImportError:
    WINDOWS_AVAILABLE = False
    print("Windows API kütüphaneleri bulunamadı. pip install pywin32 pillow komutunu çalıştırın.")

class WindowScreenshotTool:
    def __init__(self):
        self.is_running = False
        self.screenshot_thread = None
        self.window_title = ""
        self.window_hwnd = None
        self.interval = 5  # saniye
        self.save_folder = "screenshots"
        self.background_mode = True  # Arka plan pencere screenshot modu

        # Kayıt klasörü oluştur
        if not os.path.exists(self.save_folder):
            os.makedirs(self.save_folder)

    def pil_to_cv(self, pil_image):
        """PIL → OpenCV (BGR)"""
        return cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

    def detect_and_save_faces(self, image, base_filename, save_folder="faces"):
        """Yüzleri bul ve kaydet"""
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(
            gray, scaleFactor=1.3, minNeighbors=5, minSize=(96,96)
        )

        # Kare çiz (istersen kaydetme kısmını bırakabilirsin)
        if len(faces) == 0:
                print("Hiç yüz bulunamadı.")
        else:
            for (x, y, w, h) in faces:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            face_files = []
            for i, (x, y, w, h) in enumerate(faces):
                face_img = image[y:y+h, x:x+w]  # yüz kesiti
                result = predict_emotion_from_image(face_img, model, le, transform_predict, device, enhance_quality=True)

                if result is not None:
                    predicted_emotion = result['predicted_emotion']
                    confidence = result['confidence']

                    # Yüze tahmin yaz (BGR formatında)
                    cv2.putText(face_img, f"{predicted_emotion} ({confidence*100:.1f}%)",
                                (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                    # Kayıt et
                    filename = os.path.join("faces", f"face_{i}_{predicted_emotion}.png")
                    cv2.imwrite(filename, face_img)
                    face_files.append(filename)

        print("Tahmin edilen yüzler kaydedildi:", face_files)

    def list_windows(self):
        """Windows'taki tüm pencereleri listele"""
        if not WINDOWS_AVAILABLE:
            return []

        windows = []

        def enum_windows_callback(hwnd, windows_list):
            if win32gui.IsWindowVisible(hwnd):
                window_text = win32gui.GetWindowText(hwnd)
                if window_text and len(window_text) > 0:
                    # Pencere boyutlarını kontrol et (çok küçük pencereleri filtrele)
                    rect = win32gui.GetWindowRect(hwnd)
                    width = rect[2] - rect[0]
                    height = rect[3] - rect[1]
                    if width > 50 and height > 50:  # Minimum boyut kontrolü
                        windows_list.append((hwnd, window_text))

        win32gui.EnumWindows(enum_windows_callback, windows)
        return windows

    def capture_window_background(self, hwnd):
        """Arka planda olan pencereyi görünür hale getirmeden screenshot al"""
        try:
            # Pencere boyutlarını al
            rect = win32gui.GetWindowRect(hwnd)
            x, y, x1, y1 = rect
            width = x1 - x
            height = y1 - y

            if width <= 0 or height <= 0:
                print("Geçersiz pencere boyutları!")
                return None

            # Pencere DC'sini al
            hwndDC = win32gui.GetWindowDC(hwnd)
            mfcDC = win32ui.CreateDCFromHandle(hwndDC)
            saveDC = mfcDC.CreateCompatibleDC()

            # Bitmap oluştur
            saveBitMap = win32ui.CreateBitmap()
            saveBitMap.CreateCompatibleBitmap(mfcDC, width, height)
            saveDC.SelectObject(saveBitMap)

            # Pencereyi bitmap'e çiz
            result = windll.user32.PrintWindow(hwnd, saveDC.GetSafeHdc(), 3)

            if result == 0:
                print("PrintWindow başarısız oldu, alternatif yöntem deneniyor...")
                # Alternatif: Pencereyi kısa süre öne getir
                return self.capture_window_foreground(hwnd)

            # Bitmap verilerini al
            bmpinfo = saveBitMap.GetInfo()
            bmpstr = saveBitMap.GetBitmapBits(True)

            # PIL Image'a dönüştür
            im = Image.frombuffer(
                'RGB',
                (bmpinfo['bmWidth'], bmpinfo['bmHeight']),
                bmpstr, 'raw', 'BGRX', 0, 1)

            # Temizlik
            win32gui.DeleteObject(saveBitMap.GetHandle())
            saveDC.DeleteDC()
            mfcDC.DeleteDC()
            win32gui.ReleaseDC(hwnd, hwndDC)

            return im

        except Exception as e:
            print(f"Arka plan screenshot hatası: {e}")
            return self.capture_window_foreground(hwnd)

    def capture_window_foreground(self, hwnd):
        """Pencereyi öne getirerek screenshot al (yedek yöntem)"""
        try:
            # Mevcut aktif pencereyi hatırla
            current_hwnd = win32gui.GetForegroundWindow()

            # Hedef pencereyi öne getir
            win32gui.ShowWindow(hwnd, win32con.SW_RESTORE)
            win32gui.SetForegroundWindow(hwnd)
            time.sleep(0.5)  # Pencere odaklanması için bekle

            # Pencere koordinatlarını al
            rect = win32gui.GetWindowRect(hwnd)
            x, y, x1, y1 = rect

            # Screenshot al
            screenshot = pyautogui.screenshot(region=(x, y, x1-x, y1-y))

            # Önceki pencereyi geri getir
            if current_hwnd:
                win32gui.SetForegroundWindow(current_hwnd)

            return screenshot

        except Exception as e:
            print(f"Ön plan screenshot hatası: {e}")
            return None

    def get_window_screenshot(self, window_title=None, hwnd=None):
        """Belirtilen pencereden screenshot al"""
        if not WINDOWS_AVAILABLE:
            return pyautogui.screenshot()

        # HWND kullan veya pencere başlığından bul
        if hwnd:
            target_hwnd = hwnd
        else:
            target_hwnd = win32gui.FindWindow(None, window_title)

        if not target_hwnd:
            print(f"'{window_title}' penceresi bulunamadı!")
            return None

        # Pencere durumunu kontrol et
        if not win32gui.IsWindow(target_hwnd):
            print("Geçersiz pencere handle'ı!")
            return None

        if self.background_mode:
            # Arka plan screenshot (pencereyi öne getirmeden)
            print("Arka plan screenshot alınıyor...")
            screenshot = self.capture_window_background(target_hwnd)
        else:
            # Normal screenshot (pencereyi öne getirerek)
            print("Normal screenshot alınıyor...")
            screenshot = self.capture_window_foreground(target_hwnd)

        return screenshot

    def save_screenshot(self, screenshot, window_title):
        """Screenshot'ı dosyaya kaydet"""
        if screenshot:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            clean_title = "".join(c for c in window_title if c.isalnum() or c in (' ', '-', '_')).rstrip()
            clean_title = clean_title.replace(' ', '_')
            filename = f"{clean_title}_{timestamp}.png"
            filepath = os.path.join(self.save_folder, filename)

            # Screenshot kaydet
            screenshot.save(filepath)
            print(f"Screenshot kaydedildi: {filepath}")

            # OpenCV formatına çevir
            cv_img = self.pil_to_cv(screenshot)
            self.detect_and_save_faces(cv_img, f"{clean_title}_{timestamp}")

            return filepath
        return None

    def screenshot_loop(self):
        """Ana screenshot alma döngüsü"""
        print(f"Screenshot alma başladı.")
        print(f"Pencere: '{self.window_title}'")
        print(f"Aralık: {self.interval} saniye")
        print(f"Mod: {'Arka Plan' if self.background_mode else 'Ön Plan'}")

        while self.is_running:
            screenshot = self.get_window_screenshot(self.window_title, self.window_hwnd)
            if screenshot:
                self.save_screenshot(screenshot, self.window_title)
            else:
                print("Screenshot alınamadı!")

            time.sleep(self.interval)

    def start_screenshots(self):
        """Screenshot almayı başlat"""
        if not self.is_running:
            self.is_running = True
            self.screenshot_thread = threading.Thread(target=self.screenshot_loop)
            self.screenshot_thread.daemon = True
            self.screenshot_thread.start()

    def stop_screenshots(self):
        """Screenshot almayı durdur"""
        self.is_running = False
        if self.screenshot_thread:
            self.screenshot_thread.join(timeout=1)

def create_gui():
    """Gelişmiş GUI oluştur"""
    tool = WindowScreenshotTool()

    root = tk.Tk()
    root.title("Gelişmiş Pencere Screenshot Alıcı")
    root.geometry("600x500")

    # Mod seçimi
    mode_frame = tk.Frame(root)
    mode_frame.pack(pady=10)

    mode_var = tk.BooleanVar(value=True)
    mode_check = tk.Checkbutton(mode_frame,
                               text="Arka Plan Modu (Pencereyi öne getirmeden screenshot al)",
                               variable=mode_var,
                               font=("Arial", 10, "bold"),
                               fg="blue")
    mode_check.pack()

    # Pencere listesi
    list_frame = tk.Frame(root)
    list_frame.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

    tk.Label(list_frame, text="Mevcut Pencereler:", font=("Arial", 10, "bold")).pack(anchor=tk.W)

    # Scrollbar ile listbox
    scrollbar = tk.Scrollbar(list_frame)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    windows_listbox = tk.Listbox(list_frame, height=12, yscrollcommand=scrollbar.set)
    windows_listbox.pack(fill=tk.BOTH, expand=True)
    scrollbar.config(command=windows_listbox.yview)

    # Pencere verileri
    windows_data = []

    def refresh_windows():
        """Pencere listesini yenile"""
        nonlocal windows_data
        windows_listbox.delete(0, tk.END)

        if WINDOWS_AVAILABLE:
            windows_data = tool.list_windows()
            for hwnd, title in windows_data:
                # Pencere durumu bilgisini ekle
                try:
                    if win32gui.IsIconic(hwnd):
                        status = " [Minimize]"
                    elif win32gui.GetForegroundWindow() == hwnd:
                        status = " [Aktif]"
                    else:
                        status = " [Arka Plan]"

                    display_text = f"{title}{status}"
                    windows_listbox.insert(tk.END, display_text)
                except:
                    windows_listbox.insert(tk.END, title)
        else:
            windows_listbox.insert(tk.END, "Windows API bulunamadı - Tam ekran screenshot")

    def start_capture():
        """Screenshot almayı başlat"""
        try:
            # Mod ayarını güncelle
            tool.background_mode = mode_var.get()

            if WINDOWS_AVAILABLE:
                selection = windows_listbox.curselection()
                if not selection:
                    messagebox.showwarning("Uyarı", "Lütfen bir pencere seçin!")
                    return

                selected_index = selection[0]
                tool.window_hwnd, tool.window_title = windows_data[selected_index]
            else:
                tool.window_title = "FullScreen"
                tool.window_hwnd = None

            # Zaman aralığını al
            interval = simpledialog.askinteger("Zaman Aralığı",
                                             "Screenshot aralığını saniye cinsinden girin:",
                                             initialvalue=5, minvalue=1, maxvalue=3600)
            if interval:
                tool.interval = interval
                tool.start_screenshots()
                start_btn.config(state=tk.DISABLED)
                stop_btn.config(state=tk.NORMAL)
                mode_check.config(state=tk.DISABLED)

                mode_text = "Arka Plan Modu" if tool.background_mode else "Ön Plan Modu"
                messagebox.showinfo("Başlatıldı",
                                   f"Screenshot alma başladı!\n"
                                   f"Mod: {mode_text}\n"
                                   f"Pencere: {tool.window_title}\n"
                                   f"Klasör: {tool.save_folder}")

        except Exception as e:
            messagebox.showerror("Hata", f"Bir hata oluştu: {str(e)}")

    def stop_capture():
        """Screenshot almayı durdur"""
        tool.stop_screenshots()
        start_btn.config(state=tk.NORMAL)
        stop_btn.config(state=tk.DISABLED)
        mode_check.config(state=tk.NORMAL)
        messagebox.showinfo("Durduruldu", "Screenshot alma durduruldu!")

    def test_screenshot():
        """Tek screenshot al (test için)"""
        try:
            tool.background_mode = mode_var.get()

            if WINDOWS_AVAILABLE:
                selection = windows_listbox.curselection()
                if not selection:
                    messagebox.showwarning("Uyarı", "Lütfen bir pencere seçin!")
                    return

                selected_index = selection[0]
                hwnd, title = windows_data[selected_index]
                screenshot = tool.get_window_screenshot(title, hwnd)

                if screenshot:
                    filepath = tool.save_screenshot(screenshot, title)
                    messagebox.showinfo("Başarılı", f"Test screenshot kaydedildi:\n{filepath}")
                else:
                    messagebox.showerror("Hata", "Screenshot alınamadı!")

        except Exception as e:
            messagebox.showerror("Hata", f"Test screenshot hatası: {str(e)}")

    # Butonlar
    button_frame = tk.Frame(root)
    button_frame.pack(pady=15)

    refresh_btn = tk.Button(button_frame, text="🔄 Pencereleri Yenile",
                           command=refresh_windows, font=("Arial", 9))
    refresh_btn.pack(side=tk.LEFT, padx=5)

    test_btn = tk.Button(button_frame, text="📸 Test Screenshot",
                        command=test_screenshot, font=("Arial", 9))
    test_btn.pack(side=tk.LEFT, padx=5)

    start_btn = tk.Button(button_frame, text="▶️ Otomatik Başlat",
                         command=start_capture, bg="green", fg="white", font=("Arial", 9, "bold"))
    start_btn.pack(side=tk.LEFT, padx=5)

    stop_btn = tk.Button(button_frame, text="⏹️ Durdur",
                        command=stop_capture, bg="red", fg="white", font=("Arial", 9, "bold"),
                        state=tk.DISABLED)
    stop_btn.pack(side=tk.LEFT, padx=5)

    # Bilgi paneli
    info_frame = tk.Frame(root)
    info_frame.pack(pady=10, padx=10, fill=tk.X)

    info_text = f"📁 Screenshot'lar '{tool.save_folder}' klasörüne kaydedilecek.\n"

    if WINDOWS_AVAILABLE:
        info_text += "✅ Windows API mevcut - Gelişmiş pencere kontrolü aktif.\n"
        info_text += "🔵 Arka Plan Modu: Pencereyi öne getirmeden screenshot alır.\n"
        info_text += "🔴 Normal Mod: Pencereyi kısa süre öne getirerek screenshot alır."
    else:
        info_text += "⚠️ Windows API bulunamadı. 'pip install pywin32 pillow' komutunu çalıştırın."

    info_label = tk.Label(info_frame, text=info_text, wraplength=550, justify=tk.LEFT,
                         font=("Arial", 8), bg="#f0f0f0", padx=10, pady=5)
    info_label.pack(fill=tk.X)

    # İlk yükleme
    refresh_windows()

    root.mainloop()

if __name__ == "__main__":
    # Gerekli kütüphaneleri kontrol et
    try:
        import pyautogui
        pyautogui.FAILSAFE = False
    except ImportError:
        print("❌ pyautogui kütüphanesi bulunamadı.")
        print("📥 Lütfen şu komutu çalıştırın: pip install pyautogui pillow pywin32")
        exit(1)

    if not WINDOWS_AVAILABLE:
        print("⚠️ Windows API kütüphaneleri bulunamadı.")
        print("📥 Tam özellik için şu komutu çalıştırın: pip install pywin32 pillow")
        print("💡 Şimdilik temel özelliklerle devam ediliyor...\n")

    print("🚀 Gelişmiş Pencere Screenshot Alıcı")
    print("=" * 40)



    create_gui()

🚀 Gelişmiş Pencere Screenshot Alıcı
Screenshot alma başladı.
Pencere: 'human faces - Google Search - Google Chrome'
Aralık: 5 saniye
Mod: Arka Plan
Arka plan screenshot alınıyor...
Screenshot kaydedildi: screenshots\human_faces_-_Google_Search_-_Google_Chrome_20250826_153518.png
Tahmin edilen yüzler kaydedildi: ['faces\\face_0_anger.png', 'faces\\face_1_anger.png', 'faces\\face_2_surprise.png', 'faces\\face_3_happiness.png', 'faces\\face_4_surprise.png', 'faces\\face_5_anger.png', 'faces\\face_6_happiness.png', 'faces\\face_7_anger.png', 'faces\\face_8_surprise.png', 'faces\\face_9_surprise.png', 'faces\\face_10_surprise.png', 'faces\\face_11_surprise.png', 'faces\\face_12_neutrality.png']
Arka plan screenshot alınıyor...
Screenshot kaydedildi: screenshots\human_faces_-_Google_Search_-_Google_Chrome_20250826_153524.png
Tahmin edilen yüzler kaydedildi: ['faces\\face_0_anger.png', 'faces\\face_1_anger.png', 'faces\\face_2_surprise.png', 'faces\\face_3_happiness.png', 'faces\\face_4_surpr

In [12]:
import os

from collections import Counter

# Yüzlerin kaydedildiği klasör
faces_folder = "faces"

# Dosya isimlerinden duyguları al
emotion_labels = []
for filename in os.listdir(faces_folder):
    if filename.endswith(".png"):
        # filename örnek: face_0_happiness.png
        parts = filename.split("_")
        if len(parts) >= 3:
            emotion = parts[-1].replace(".png", "")  # son kısım duygu
            emotion_labels.append(emotion)

# Duygu dağılımını hesapla
emotion_counts = Counter(emotion_labels)
total_faces = sum(emotion_counts.values())

# Yüzdeyi hesapla ve yazdır
print("Duygu Dağılımı:")
for emotion, count in emotion_counts.items():
    print(f"{emotion}: {count} ({count/total_faces*100:.2f}%)")

Duygu Dağılımı:
anger: 6 (31.58%)
surprise: 9 (47.37%)
neutrality: 1 (5.26%)
happiness: 3 (15.79%)


In [ ]:
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
from torchvision import transforms
import cv2
import numpy as np
from PIL import Image, ImageFile
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import albumentations as A
from imblearn.over_sampling import RandomOverSampler
from tqdm import tqdm
import os
from datetime import datetime

# Truncated image loading için
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Device selection
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan cihaz: {device}")

# Veri yükleme
labels = []
file_names = []
base_path = Path(r"C:\Users\mibra\Downloads\archive (4)\fer_ckplus_kdef")

for file in sorted(base_path.rglob('*.*')):
    label = file.parent.name
    if label not in ["contempt", "fear", "disgust"]:  # Temizlenmiş kontrol
        labels.append(label)
        file_names.append(str(file))

print(f"Toplam dosya: {len(file_names)}, Etiket sayısı: {len(labels)}")

df = pd.DataFrame({"image": file_names, "label": labels})
print(f"DataFrame shape: {df.shape}")
print("Sınıf dağılımı:", Counter(labels))

y = df["label"].tolist()
x = df["image"].tolist()

from collections import Counter
import numpy as np
import cv2
import albumentations as A
from tqdm import tqdm
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from collections import Counter
import numpy as np
import cv2
import albumentations as A
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler

def balance_with_augmentation_optimized(X_paths, y_labels, img_size=(224, 224)):
    """Bütün sınıfları mean seviyesine eşitleme (undersample + augmentation oversample)"""

    # Albumentations augmentations
    aug_transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=20, p=0.6),
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
        A.Affine(
            translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
            scale={"x": (0.85, 1.15), "y": (0.85, 1.15)},
            rotate=(-15, 15),
            p=0.5
        ),
    ])

    class_counts = Counter(y_labels)
    mean_count = int(np.mean(list(class_counts.values())))

    print(f"Orijinal sınıf dağılımı: {class_counts}")
    print(f"Hedef sınıf sayısı (mean-based): {mean_count}")

    # Resim + label depoları
    X_dict = {cls: [] for cls in class_counts.keys()}

    # Görselleri oku ve enhance et
    for idx, path in enumerate(tqdm(X_paths)):
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        img = cv2.resize(img, img_size)

        # Enhancement (sharpen + CLAHE)
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        img = cv2.filter2D(img, -1, kernel)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        img = clahe.apply(img)

        label = y_labels[idx]
        X_dict[label].append(img)

    # Final balanced data
    X_final, y_final = [], []

    # Her sınıf için balancing
    for label, imgs in X_dict.items():
        count = len(imgs)

        if count > mean_count:
            # Fazla sınıflar → undersample
            selected = np.random.choice(range(count), mean_count, replace=False)
            balanced_imgs = [imgs[i] for i in selected]

        elif count < mean_count:
            balanced_imgs = imgs.copy()
            needed = mean_count - count

            while len(balanced_imgs) < mean_count:
                img = imgs[np.random.randint(0, count)]
                img_3ch = np.stack([img, img, img], axis=-1)
                augmented = aug_transform(image=img_3ch)["image"]
                augmented_gray = cv2.cvtColor(augmented, cv2.COLOR_RGB2GRAY)
                augmented_gray = cv2.resize(augmented_gray, img_size)

                # Enhancement tekrar
                augmented_gray = cv2.filter2D(augmented_gray, -1, kernel)
                augmented_gray = clahe.apply(augmented_gray)

                balanced_imgs.append(augmented_gray)

        else:
            balanced_imgs = imgs  # zaten eşit

        X_final.extend(balanced_imgs)
        y_final.extend([label] * mean_count)

    # Numpy array’e çevir
    X_final = np.array(X_final, dtype=np.uint8)
    y_final = np.array(y_final)

    print(f"Final balanced dataset: {len(X_final)} samples")
    print(f"Final sınıf dağılımı: {Counter(y_final)}")

    return X_final, y_final



# Veri hazırlama
X_final, y_final = balance_with_augmentation_optimized(x, y)

# Label encoding
le = LabelEncoder()
y_encoded = le.fit_transform(y_final)

# Label encoder'ı kaydet
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

print(f"Label classes: {le.classes_}")
print(f"Total samples after balancing: {len(y_encoded)}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Train samples: {len(X_train)}, Test samples: {len(X_test)}")

class EmotionDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img = self.X[idx].astype(np.uint8)
        label = self.y[idx]

        try:
            img_pil = Image.fromarray(img, mode='L')

            if self.transform:
                img_tensor = self.transform(img_pil)
            else:
                img_tensor = transforms.ToTensor()(img_pil)

        except Exception as e:
            print(f"Transform hatası: {e}")
            img_tensor = torch.zeros((3, 224, 224))

        return img_tensor, torch.tensor(label, dtype=torch.long)

# Transforms
transform_train = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Datasets and DataLoaders
train_dataset = EmotionDataset(X_train, y_train, transform=transform_train)
test_dataset = EmotionDataset(X_test, y_test, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

print(f"Train batches: {len(train_loader)}, Test batches: {len(test_loader)}")

# Model oluşturma
model = models.mobilenet_v3_small(pretrained=True)

# Classifier'ı sınıf sayısına göre ayarla - BU ÖNEMLİ!
num_classes = len(le.classes_)
model.classifier[3] = nn.Linear(in_features=1024, out_features=num_classes)

# Model'i device'a taşı
model = model.to(device)

print(f"Model output classes: {num_classes}")
print(f"Model classes: {le.classes_}")

# Loss ve optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.7)

def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs=10):
    """Gelişmiş training fonksiyonu"""
    best_acc = 0.0
    train_losses = []
    test_accuracies = []

    # Training başlangıç zamanı
    start_time = datetime.now()
    print(f"Training başlangıç zamanı: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        print("-" * 40)

        for batch_idx, (inputs, labels) in enumerate(tqdm(train_loader, desc="Training")):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        train_acc = 100 * correct_train / total_train
        train_losses.append(epoch_loss)

        # Validation phase
        model.eval()
        correct_test = 0
        total_test = 0
        test_loss = 0.0

        with torch.no_grad():
            for inputs, labels in tqdm(test_loader, desc="Testing"):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

        test_acc = 100 * correct_test / total_test
        test_accuracies.append(test_acc)

        print(f"Train Loss: {epoch_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Test Loss: {test_loss / len(test_loader):.4f}, Test Acc: {test_acc:.2f}%")

        # Best model kaydetme
        if test_acc > best_acc:
            best_acc = test_acc
            # Zaman damgalı model kaydı
            timestamp = datetime.now().strftime("%Y%m%d_%H%M")
            model_path = f'best_emotion_model_{timestamp}.pth'
            torch.save(model.state_dict(), model_path)
            print(f"New best accuracy: {best_acc:.2f}% - Model saved: {model_path}")

        scheduler.step()
        print(f"Learning rate: {scheduler.get_last_lr()[0]:.6f}")

    end_time = datetime.now()
    training_duration = end_time - start_time
    print(f"\nTraining completed!")
    print(f"Training duration: {training_duration}")
    print(f"Best test accuracy: {best_acc:.2f}%")

    return train_losses, test_accuracies, best_acc

# Model training
print("Model training başlıyor...")
train_losses, test_accuracies, best_accuracy = train_model(
    model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs=10
)

# Son modeli de kaydet
final_timestamp = datetime.now().strftime("%Y%m%d_%H%M")
final_model_path = f'final_emotion_model_{final_timestamp}.pth'
torch.save(model.state_dict(), final_model_path)
print(f"Final model saved: {final_model_path}")

# Metadata kaydetme
model_info = {
    'model_state_dict': model.state_dict(),
    'num_classes': num_classes,
    'class_names': le.classes_.tolist(),
    'best_accuracy': best_accuracy,
    'training_date': datetime.now().isoformat(),
    'model_architecture': 'MobileNetV3-Small',
    'input_size': (224, 224),
    'final_train_loss': train_losses[-1] if train_losses else None,
    'final_test_accuracy': test_accuracies[-1] if test_accuracies else None
}

metadata_path = f'model_info_{final_timestamp}.pkl'
with open(metadata_path, 'wb') as f:
    pickle.dump(model_info, f)
print(f"Model metadata saved: {metadata_path}")

# Final evaluation
print("\n" + "="*50)
print("FINAL EVALUATION")
print("="*50)

model.eval()
correct = 0
total = 0
class_correct = [0] * num_classes
class_total = [0] * num_classes

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Final Evaluation"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Per-class accuracy
        for i in range(labels.size(0)):
            label_idx = labels[i].item()
            pred_idx = predicted[i].item()

            class_total[label_idx] += 1
            if label_idx == pred_idx:
                class_correct[label_idx] += 1

final_accuracy = 100 * correct / total
print(f'Final Test Accuracy: {final_accuracy:.2f}%')
print(f'Total test samples: {total}')

# Class-wise accuracy
print(f"\nClass-wise Accuracies:")
print("-" * 50)
for i, class_name in enumerate(le.classes_):
    if class_total[i] > 0:
        class_acc = 100 * class_correct[i] / class_total[i]
        print(f'{class_name:>12}: {class_acc:>6.2f}% ({class_correct[i]:>4}/{class_total[i]:>4})')
    else:
        print(f'{class_name:>12}: No samples in test set')

# Detailed classification report ve confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Tüm predictions topla
all_predictions = []
all_labels = []

model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Classification report
print(f"\nDetailed Classification Report:")
print("-" * 60)
report = classification_report(all_labels, all_predictions,
                               target_names=le.classes_, digits=3)
print(report)

# Confusion Matrix
plt.figure(figsize=(10, 8))
cm = confusion_matrix(all_labels, all_predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title(f'Confusion Matrix - Accuracy: {final_accuracy:.2f}%')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()

# Confusion matrix kaydet
cm_path = f'confusion_matrix_{final_timestamp}.png'
plt.savefig(cm_path, dpi=300, bbox_inches='tight')
plt.show()

print(f"Confusion matrix saved: {cm_path}")

# Final summary
print(f"\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)
print(f"Best model saved as: best_emotion_model_{final_timestamp}.pth")
print(f"Final model saved as: {final_model_path}")
print(f"Metadata saved as: {metadata_path}")
print(f"Confusion matrix: {cm_path}")
print(f"Best accuracy achieved: {best_accuracy:.2f}%")
print(f"Final test accuracy: {final_accuracy:.2f}%")
print(f"Number of classes: {num_classes}")
print(f"Classes: {', '.join(le.classes_)}")
print("="*60)

# Prediction function
import torch.nn.functional as F

def predict_emotion_from_path(image_path, model, le, transform, device, show_image=True, show_probabilities=True):
    """
    Tek resim yolu kullanarak emotion prediction yapar

    Args:
        image_path (str): Resim dosyasının yolu
        model: Eğitilmiş PyTorch modeli
        le: LabelEncoder objesi
        transform: Preprocessing transform'ları
        device: PyTorch device (cuda/cpu)
        show_image (bool): Resmi göster/gösterme
        show_probabilities (bool): Tüm sınıfların olasılıklarını göster

    Returns:
        dict: Prediction sonuçları
    """
    try:
        # Resmi yükle ve kontrol et
        if not os.path.exists(image_path):
            print(f"Hata: {image_path} dosyası bulunamadı!")
            return None

        # PIL ile resmi aç
        image = Image.open(image_path)
        original_image = image.copy()  # Gösterim için orijinal resmi sakla

        # Resmi göster (opsiyonel)
        if show_image:
            plt.figure(figsize=(6, 4))
            plt.imshow(image, cmap='gray' if image.mode == 'L' else None)
            plt.title(f"Input Image: {os.path.basename(image_path)}")
            plt.axis('off')
            plt.show()

        # Model'i evaluation moduna al
        model.eval()

        # Resmi transform et
        if transform:
            input_tensor = transform(image).unsqueeze(0)  # Batch dimension ekle
        else:
            # Fallback transform
            input_tensor = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.Grayscale(num_output_channels=3),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])(image).unsqueeze(0)

        # GPU'ya taşı
        input_tensor = input_tensor.to(device)

        # Prediction yap
        with torch.no_grad():
            outputs = model(input_tensor)
            probabilities = F.softmax(outputs, dim=1)

        # En yüksek olasılık ve sınıf
        max_prob, predicted_class = torch.max(probabilities, 1)
        predicted_emotion = le.classes_[predicted_class.item()]
        confidence = max_prob.item()

        # Tüm sınıfların olasılıkları
        all_probabilities = probabilities[0].cpu().numpy()
        emotion_probabilities = {}

        for i, emotion in enumerate(le.classes_):
            emotion_probabilities[emotion] = all_probabilities[i]

        # Sonuçları sırala (en yüksekten en düşüğe)
        sorted_emotions = sorted(emotion_probabilities.items(), key=lambda x: x[1], reverse=True)

        # Sonuçları yazdır
        print(f"\n{'='*60}")
        print(f"EMOTION PREDICTION RESULTS")
        print(f"{'='*60}")
        print(f"Image: {os.path.basename(image_path)}")
        print(f"Predicted Emotion: {predicted_emotion.upper()}")
        print(f"Confidence: {confidence*100:.2f}%")
        print(f"Device: {device}")

        if show_probabilities:
            print(f"\nAll Emotion Probabilities:")
            print(f"{'-'*40}")
            for emotion, prob in sorted_emotions:
                bar = "█" * int(prob * 20)  # Simple progress bar
                print(f"{emotion:>12}: {prob*100:>6.2f}% {bar}")

        # Return dictionary
        result = {
            'predicted_emotion': predicted_emotion,
            'confidence': confidence,
            'all_probabilities': emotion_probabilities,
            'sorted_probabilities': sorted_emotions,
            'image_path': image_path
        }

        return result

    except Exception as e:
        print(f"Prediction hatası: {e}")
        return None

# Transform for prediction
transform_predict = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Example usage:
# image_path = r"path/to/your/image.jpg"
# result = predict_emotion_from_path(image_path, model, le, transform_predict, device)